In [1]:
!pip install autogluon
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from scipy.stats import randint
import numpy as np

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ | done
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 6.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 10.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
INFO: pip is looking at multiple versions of openxlab to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of openxlab to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than

In [2]:
!pip uninstall lightgbm -y
!pip install lightgbm --no-cache-dir



Found existing installation: lightgbm 4.2.0
Uninstalling lightgbm-4.2.0:
  Successfully uninstalled lightgbm-4.2.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 17.4 MB/s eta 0:00:00


In [3]:
import pandas as pd
from autogluon.tabular import TabularDataset, TabularPredictor

# Load data
train_df = pd.read_csv('/kaggle/input/playground-series-s4e9/train.csv')
test_df = pd.read_csv('/kaggle/input/playground-series-s4e9/test.csv')

# Display first few rows
train_df.head()
test_df.head()


,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title
0,188533,Land,Rover LR2 Base,2015,98000,Gasoline,240.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,6-Speed A/T,White,Beige,None reported,Yes
1,188534,Land,Rover Defender SE,2020,9142,Hybrid,395.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,8-Speed A/T,Silver,Black,None reported,Yes
2,188535,Ford,Expedition Limited,2022,28121,Gasoline,3.5L V6 24V PDI DOHC Twin Turbo,10-Speed Automatic,White,Ebony,None reported,NaN
3,188536,Audi,A6 2.0T Sport,2016,61258,Gasoline,2.0 Liter TFSI,Automatic,Silician Yellow,Black,None reported,NaN
4,188537,Audi,A6 2.0T Premium Plus,2018,59000,Gasoline,252.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,A/T,Gray,Black,None reported,Yes


In [4]:
import re
# Extract horsepower and engine size from the 'engine' column
def extract_engine_features(engine_str):
    hp = None
    size = None
    
    # Match the horsepower
    hp_match = re.search(r"(\d+(\.\d+)?)HP", engine_str)
    if hp_match:
        hp = float(hp_match.group(1))
    
    # Match the engine size
    size_match = re.search(r"(\d+(\.\d+)?)L", engine_str)
    if size_match:
        size = float(size_match.group(1))
    
    return pd.Series([hp, size])

# Apply the function to train and test data
train_df[['engine_hp', 'engine_size']] = train_df['engine'].apply(extract_engine_features)
test_df[['engine_hp', 'engine_size']] = test_df['engine'].apply(extract_engine_features)

# Drop the 'engine' column after feature extraction
train_df = train_df.drop(columns=['engine'])
test_df = test_df.drop(columns=['engine'])

In [5]:
# Handle missing values in train_df (including the 'price' column)
for col in train_df.select_dtypes(include=['float64', 'int64']).columns:
    train_df[col].fillna(train_df[col].mean(), inplace=True)

# Handle missing values in test_df (excluding the 'price' column, since it doesn't exist)
for col in test_df.select_dtypes(include=['float64', 'int64']).columns:
    test_df[col].fillna(test_df[col].mean(), inplace=True)

# Fill missing values in categorical columns with the most frequent value for both train_df and test_df
for col in train_df.select_dtypes(include=['object']).columns:
    train_df[col].fillna(train_df[col].mode()[0], inplace=True)

for col in test_df.select_dtypes(include=['object']).columns:
    test_df[col].fillna(test_df[col].mode()[0], inplace=True)

# Convert categorical columns to category dtype
for col in train_df.select_dtypes(include=['object']).columns:
    train_df[col] = train_df[col].astype('category')
    test_df[col] = test_df[col].astype('category')


/tmp/ipykernel_22/3471104084.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_df[col].fillna(train_df[col].mean(), inplace=True)
/tmp/ipykernel_22/3471104084.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', 

In [6]:
# Feature Engineering

# Create interaction features
train_df['hp_per_liter'] = train_df['engine_hp'] / train_df['engine_size']
test_df['hp_per_liter'] = test_df['engine_hp'] / test_df['engine_size']

# Convert 'model_year' to age of the car
current_year = 2024
train_df['car_age'] = current_year - train_df['model_year']
test_df['car_age'] = current_year - test_df['model_year']

# Encode categorical features using one-hot encoding
categorical_cols = ['brand', 'model', 'fuel_type', 'transmission', 'ext_col', 'int_col']
train_df = pd.get_dummies(train_df, columns=categorical_cols, drop_first=True)
test_df = pd.get_dummies(test_df, columns=categorical_cols, drop_first=True)

# Ensure the train and test data have the same columns after one-hot encoding
train_df, test_df = train_df.align(test_df, join='left', axis=1, fill_value=0)

# Ensure that newly created columns do not have missing values
train_df['hp_per_liter'].fillna(train_df['hp_per_liter'].mean(), inplace=True)
test_df['hp_per_liter'].fillna(test_df['hp_per_liter'].mean(), inplace=True)
train_df['car_age'].fillna(train_df['car_age'].mean(), inplace=True)
test_df['car_age'].fillna(test_df['car_age'].mean(), inplace=True)


/tmp/ipykernel_22/1757476192.py:21: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_df['hp_per_liter'].fillna(train_df['hp_per_liter'].mean(), inplace=True)
/tmp/ipykernel_22/1757476192.py:22: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method

In [7]:
predictor = TabularPredictor(label='price', eval_metric='rmse').fit(
    train_df,
    presets='best_quality',
    time_limit=32000,  # 9 hours minus buffer
    verbosity=2,  # Moderate output
    excluded_model_types=['KNN', 'LR'],  # Exclude models with poor performance
    ag_args_fit={'num_gpus': 2}
)


No path specified. Models will be saved in: "AutogluonModels/ag-20240904_010311"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.14
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Thu Jun 27 20:43:36 UTC 2024
CPU Count:          4
Memory Avail:       29.19 GB / 31.36 GB (93.1%)
Disk Space Avail:   19.50 GB / 19.52 GB (99.9%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon will be fit on subsets o

In [8]:
# Print model performance
results = predictor.fit_summary()
print(results)

# Get leaderboard
lb = predictor.leaderboard()
print(lb)


*** Summary of fit() ***
Estimated performance of each model:
                    model     score_val              eval_metric  pred_time_val      fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0     WeightedEnsemble_L3 -72761.245572  root_mean_squared_error     355.381454  21594.292533                0.003317           0.298644            3       True         10
1     WeightedEnsemble_L2 -72812.436591  root_mean_squared_error       7.106460    353.907629                0.003175           0.212070            2       True          6
2       LightGBMXT_BAG_L2 -72869.153464  root_mean_squared_error     235.447983  12186.515690                2.912507         116.216893            2       True          7
3         LightGBM_BAG_L1 -72875.091207  root_mean_squared_error       2.924212    109.726072                2.924212         109.726072            1       True          2
4       LightGBMXT_BAG_L1 -72893.463223  root_mean_squared_error       3.42245

In [9]:
# Generate predictions for test data
test_predictions = predictor.predict(test_df)

# Create submission DataFrame
submission_df = pd.DataFrame({
    'id': test_df['id'],
    'price': test_predictions
})

# Save submission file
submission_df.to_csv('/kaggle/working/submission.csv', index=False)


In [10]:
submission_df.head()

,id,price
0,188533,18013.982422
1,188534,77154.820312
2,188535,54196.339844
3,188536,31552.570312
4,188537,30238.585938
